In [1]:
from examples import phase_music, entanglement_music_1, entanglement_music_2, entanglement_music_3, spin_temporal

pygame 2.1.3 (SDL 2.0.22, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


Example of entangled sounds

In [2]:
entanglement_music_1(1)

In [3]:
entanglement_music_2(4)

In [4]:
entanglement_music_3(4)

Examples of phases

In [5]:
phase_music(4, 3.14159265 * 2)

A simple song that mixes circuits

In [6]:
entanglement_music_2(4, 3)
entanglement_music_3(4, 3)
entanglement_music_3(4, 3)
entanglement_music_3(4, 3)
phase_music(4, 3.14159265 * 2, 3)
phase_music(4, 3.14159265 * 2, 3)
entanglement_music_2(4, 3)
entanglement_music_3(4, 3)


A simple example which uses a custom hamiltonian

In [7]:
spin_temporal(10)